<a href="https://colab.research.google.com/github/CrowdSalat/ann-dynamical-systems/blob/master/predict_plane_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# predict plane state

Uses actions and a input state to predict the outputstate.

## imports

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [13]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split


import tensorflow as tf
print(tf.__version__)
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import *

2.0.0-rc2


In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4449000914056722978
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17155272376605075971
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14396163223560150867
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11447649034033417873
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


## load files

In [0]:
file = 'dataset_all.csv'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/xplane_data' 
os.chdir(root_dir)
print ('working dir: ' + os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
working dir: /content/drive/My Drive/xplane_data


## actions & states
actions and states at time t are the input for the network.

the states at time t + 1 are the desired output of the network.


In [7]:
col_m_time      = '   missn,_time '
col_t_time      = '   _totl,_time '
col_yaw_v       = '   ____Q,rad/s '
col_heading_mag = '   hding,__mag '
col_yaw_deg     = '   _beta,__deg '
col_roll_deg    = '   _roll,__deg '
col_roll_v      = '   ____R,rad/s '
col_pitch_deg   = '   pitch,__deg '
col_pitch_v     = '   ____P,rad/s '
col_alt         = '   __alt,ftmsl '
col_vv          = '   __VVI,__fpm '
col_v           = '   Vtrue,_ktas '
col_rpm_prop    = '   rpm_1,_prop '
col_throttle     = '   thro1,_part '

cols_state = [ 
    col_yaw_v,
    col_yaw_deg,
    col_roll_deg,
    col_roll_v,
    col_pitch_deg,
    col_pitch_v,
    col_vv,
    col_alt,
    col_v,
    col_rpm_prop,
    col_throttle,
]

cols_time = [
    col_m_time,
    col_t_time,
]

col_elev_servo                  = '   _elev,servo '
col_ailrn_servo                  = '   ailrn,servo '
col_ruddr_servo                  = '   ruddr,servo '

cols_action = [
    col_elev_servo,
    col_ailrn_servo,
    col_ruddr_servo
]

# all input columns
cols_features = []
cols_features.extend(cols_state)
cols_features.extend(cols_action)

# orga column
col_dataset_index = 'dataset_index'
col_maneuver_index = 'maneuver_index'
col_maneuver_name = 'maneuver_name'
col_maneuver_datapoint_index = 'maneuver_datapoint_index'
col_maneuver_type = 'maneuver_type'


label_prefix = 't+1_'
cols_label = [label_prefix + col_state for col_state in cols_state]
print(cols_label)


['t+1_   ____Q,rad/s ', 't+1_   _beta,__deg ', 't+1_   _roll,__deg ', 't+1_   ____R,rad/s ', 't+1_   pitch,__deg ', 't+1_   ____P,rad/s ', 't+1_   __VVI,__fpm ', 't+1_   __alt,ftmsl ', 't+1_   Vtrue,_ktas ', 't+1_   rpm_1,_prop ', 't+1_   thro1,_part ']


In [8]:
def load_xplane_csv(csv_path, columns):
    return pd.read_csv(csv_path, delimiter=',', usecols = columns, 
                       float_precision='high')

df_features = load_xplane_csv(file, cols_features)

df_labels = load_xplane_csv(file, cols_label)

print(len(df_features))
print(len(df_labels))
assert(len(df_features) == len(df_labels))


380538
380538


drop nan rows in both features and in labels

In [9]:
df_nan = df_labels.isna().any(axis=1)

for i in range(0, len(df_nan)):
  if df_nan[i] == True:
    df_features.drop(index=i, inplace=True)

df_labels.dropna(inplace=True)

print(len(df_features))
print(len(df_labels))

380074
380074


In [0]:
def split_train_test(dataset):
  train_size = int(len(dataset) * 0.67)
  test_size = len(dataset) - train_size
  train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
  return train, test


In [0]:
features = df_features.values
labels = df_labels.values

#train_X, test_X = split_train_test(features)
#train_Y, test_Y = split_train_test(labels)
train_X, test_X, train_Y, test_Y  = train_test_split(features,labels, test_size=0.2, random_state=42)


In [16]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(304059, 14)
(304059, 11)
(76015, 14)
(76015, 11)


In [17]:
features_nr = train_X.shape[1]
target_nr = train_Y.shape[1]


baseline_model = keras.Sequential()

baseline_model.add(Dense(1200,input_shape=(features_nr,), 
                         kernel_initializer='he_uniform'))
baseline_model.add(BatchNormalization())
baseline_model.add(ReLU())

baseline_model.add(Dense(600,input_shape=(features_nr,)))
baseline_model.add(BatchNormalization())
baseline_model.add(ReLU())

baseline_model.add(Dense(600,input_shape=(features_nr,)))
baseline_model.add(BatchNormalization())
baseline_model.add(ReLU())

baseline_model.add(Dense(target_nr ))

baseline_model.compile(optimizer='adam',
                       loss='mean_squared_error',
                       metrics=['accuracy', 'binary_crossentropy', 'mean_squared_error'])

baseline_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1200)              18000     
_________________________________________________________________
batch_normalization (BatchNo (None, 1200)              4800      
_________________________________________________________________
re_lu (ReLU)                 (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 600)               720600    
_________________________________________________________________
batch_normalization_1 (Batch (None, 600)               2400      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 600)               3

In [47]:
baseline_history = baseline_model.fit(train_X,
                                      train_Y,
                                      epochs=100,
                                      batch_size=64,
                                      validation_data=(test_X, test_Y),
                                      verbose=2)


Train on 254649 samples, validate on 125425 samples
Epoch 1/20
254649/254649 - 27s - loss: 334831.3672 - accuracy: 0.9507 - binary_crossentropy: -7.4405e+03 - mean_squared_error: 334830.8750 - val_loss: 9014.4786 - val_accuracy: 0.9969 - val_binary_crossentropy: -8.6258e+03 - val_mean_squared_error: 9014.4766
Epoch 2/20
254649/254649 - 25s - loss: 4391.5614 - accuracy: 0.9597 - binary_crossentropy: -7.4880e+03 - mean_squared_error: 4391.5562 - val_loss: 751.8407 - val_accuracy: 0.9956 - val_binary_crossentropy: -8.6327e+03 - val_mean_squared_error: 751.8411
Epoch 3/20
254649/254649 - 25s - loss: 3151.8305 - accuracy: 0.9561 - binary_crossentropy: -7.4881e+03 - mean_squared_error: 3151.8330 - val_loss: 486.5068 - val_accuracy: 0.9999 - val_binary_crossentropy: -8.6287e+03 - val_mean_squared_error: 486.5070
Epoch 4/20
254649/254649 - 25s - loss: 3090.9815 - accuracy: 0.9554 - binary_crossentropy: -7.4881e+03 - mean_squared_error: 3090.9773 - val_loss: 277.1463 - val_accuracy: 0.9994 - va

In [0]:
baseline_model.save_weights('./checkpoints/baseline_model')
#baseline_model.load_weights('./checkpoints/baseline_model')

In [0]:
features_nr = train_X.shape[1]
target_nr = train_Y.shape[1]


features_nr = train_X.shape[1]
target_nr = train_Y.shape[1]


bigger_model = keras.Sequential()

bigger_model.add(Dense(1200,input_shape=(features_nr,), 
                         kernel_initializer='he_uniform'))
bigger_model.add(BatchNormalization())
bigger_model.add(ReLU())

bigger_model.add(Dense(1000,input_shape=(features_nr,)))
bigger_model.add(BatchNormalization())
bigger_model.add(ReLU())

bigger_model.add(Dense(800,input_shape=(features_nr,)))
bigger_model.add(BatchNormalization())
bigger_model.add(ReLU())

bigger_model.add(Dense(target_nr ))

bigger_model.compile(optimizer='adam',
                       loss='mean_squared_error',
                       metrics=['accuracy', 'binary_crossentropy', 'mean_squared_error'])

bigger_model.summary()

In [0]:
bigger_history = bigger_model.fit(train_X,
                                      train_Y,
                                      epochs=100,
                                      validation_data=(test_X, test_Y),
                                      verbose=2)


In [0]:
bigger_model.save_weights('./checkpoints/bigger_model')
#bigger_model.load_weights('./checkpoints/bigger_model')

In [0]:
features_nr = train_X.shape[1]
target_nr = train_Y.shape[1]


smaller_model = keras.Sequential()

smaller_model.add(Dense(600,input_shape=(features_nr,), 
                         kernel_initializer='he_uniform'))
smaller_model.add(BatchNormalization())
smaller_model.add(ReLU())

smaller_model.add(Dense(400,input_shape=(features_nr,)))
smaller_model.add(BatchNormalization())
smaller_model.add(ReLU())

smaller_model.add(Dense(400,input_shape=(features_nr,)))
smaller_model.add(BatchNormalization())
smaller_model.add(ReLU())

smaller_model.add(Dense(target_nr ))

smaller_model.compile(optimizer='adam',
                       loss='mean_squared_error',
                       metrics=['accuracy', 'binary_crossentropy', 'mean_squared_error'])

smaller_model.summary()

In [0]:
smaller_history = smaller_model.fit(train_X,
                                      train_Y,
                                      epochs=100,
                                      validation_data=(test_X, test_Y),
                                      verbose=2)


In [0]:
smaller_model.save_weights('./checkpoints/smaller_model')
#smaller_model.load_weights('./checkpoints/smaller_model')


In [0]:
def save_plots_before_show(diagramm_name):
  diagramm_folder = './diagramms'

  if not os.path.exists(diagramm_folder):
      os.makedirs(diagramm_folder)

  plt.savefig(diagramm_folder + '/' diagramm_name +'.pdf')
  plt.savefig(diagramm_folder + '/' diagramm_name +'.svg')
  plt.savefig(diagramm_folder + '/' diagramm_name +'.png')

def plot_history(histories, key='mea
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])
  save_plots_before_show('baseline_model_history')
  plt.show()


plot_history([('baseline', baseline_history),
              ('smaller', smaller_history),
              ('bigger', bigger_history)])
